<a href="https://colab.research.google.com/github/erdemsirel/data_science_case/blob/main/DataScinceCaseThalia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Data Source
# https://www.kaggle.com/code/juniorbueno/analyzing-credit-default/data

# Loading The Data

# Exploratory Data Analysis

# Data Processing

# Modelling